<div align="center">

# Power calcs for y_i vs. y_j tests

$$H_0: y_{LLM_i} = y_{LLM_j}$$
$$H_1: y_{LLM_i} \neq y_{LLM_j}$$

This notebook takes in paired benchmark data and calculates stats needed for power calculations. It then runs grid search over parameters to report sample size requirements under a range of assumptions. 
</div>

<div align="center">

## 0. Set-up
----

$$n = (z_{\alpha/2} + z_\beta)^2 \frac{(\omega^2 + \sigma_A^2/K_A + \sigma_B^2/K_B)}{\delta^2}$$

$$\textbf{For independent questions, either in single or multiple draws:}$$

$$\omega^2 = \text{Var}_(x_A) + \text{Var}_(x_B) - 2\text{Cov}_(x_A, x_B)$$

$$\sigma_A^2 = E[\sigma_{A,i}^2]$$

$$\sigma_B^2 = E[\sigma_{B,i}^2]$$

$$\text{where sigma captures the standard deviation among the same question repeated and would thus be 0 with no repeated questions}$$


$$\textbf{For clustered questions, either in single or multiple draws:}$$

$$\omega_{clustered}^2 = \text{Var}_{clustered}(x_A) + \text{Var}_{clustered}(x_B) - 2\text{Cov}_{clustered}(x_A, x_B)$$

$$\text{Var}_{clustered}(x_A) = \frac{1}{n}\sum_c \sum_i \sum_j (x_{A,i,c} - \bar{s}_A)(x_{A,j,c} - \bar{s}_A)$$

$$\text{Var}_{clustered}(x_B) = \frac{1}{n}\sum_c \sum_i \sum_j (x_{B,i,c} - \bar{s}_B)(x_{B,j,c} - \bar{s}_B)$$

$$\text{Cov}_{clustered}(x_A, x_B) = \frac{1}{n}\sum_c \sum_i \sum_j (x_{A,i,c} - \bar{s}_A)(x_{B,j,c} - \bar{s}_B)$$

$$\sigma_{A,clustered}^2 = \frac{1}{n}\sum_c \sum_i \sum_j \epsilon_{A,i,c}\epsilon_{A,j,c}$$

$$\sigma_{B,clustered}^2 = \frac{1}{n}\sum_c \sum_i \sum_j \epsilon_{B,i,c}\epsilon_{B,j,c}$$

</div>


<div align="center">

## 1. Define functions to calculate $\omega^2$, $\sigma^2$
----

</div>

In [ ]:
import pandas as pd
import numpy as np
from typing import Tuple, Dict, Any
from scipy import stats
from itertools import product
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def calculate_omega_squared(df: pd.DataFrame, 
                           score1_col: str = 'score1', 
                           score2_col: str = 'score2',
                           question_id_col: str = 'question_id',
                           question_draw_col: str = 'question_draw', 
                           question_type_col: str = 'question_type') -> Dict[str, Any]:
    """
    Calculate ω² adaptively based on dataset structure.
    
    Automatically detects:
    - Whether questions are clustered (multiple question_types) based on whether there is >1 unique value of question_type
    - Whether there are multiple draws per question (multiple question_draws), based on whether there is >1 unique value of question_draw
    
    Parameters:
    -----------
    df : pd.DataFrame
        Long format data with columns for question_id, question_draw, question_type, scores
    score1_col : str
        Column name for scorer 1's scores
    score2_col : str  
        Column name for scorer 2's scores
    question_id_col : str
        Column name for question identifiers
    question_draw_col : str
        Column name for question draw numbers
    question_type_col : str
        Column name for question types/clusters
        
    Returns:
    --------
    dict : Results containing omega_squared and diagnostic information
    """
    
    # Validate input data
    required_cols = [question_id_col, question_draw_col, question_type_col, score1_col, score2_col]
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")
    
    # Detect dataset characteristics
    n_question_types = df[question_type_col].nunique()
    n_draws_per_question = df.groupby(question_id_col)[question_draw_col].nunique().max()
    n_total_questions = df[question_id_col].nunique()
    
    is_clustered = n_question_types > 1
    has_multiple_draws = n_draws_per_question > 1
    
    print(f"Dataset characteristics:")
    print(f"  - Total unique questions: {n_total_questions}")
    print(f"  - Question types/clusters: {n_question_types}")
    print(f"  - Max draws per question: {n_draws_per_question}")
    print(f"  - Clustered analysis: {is_clustered}")
    print(f"  - Multiple draws: {has_multiple_draws}")
    print()
    
    # Calculate conditional means for each question (averaging across draws if multiple)
    question_means = df.groupby([question_id_col, question_type_col]).agg({
        score1_col: 'mean',
        score2_col: 'mean'
    }).reset_index()
    
    question_means.columns = [question_id_col, question_type_col, 'mean_A', 'mean_B']
    
    # Calculate overall means
    overall_mean_A = question_means['mean_A'].mean()
    overall_mean_B = question_means['mean_B'].mean()
    
    if is_clustered:
        # Use clustered variance/covariance calculations
        omega_squared = calculate_clustered_omega_squared(
            question_means, question_type_col, overall_mean_A, overall_mean_B
        )
        method = "clustered"
    else:
        # Use simple variance/covariance calculations
        var_A = np.var(question_means['mean_A'], ddof=1)
        var_B = np.var(question_means['mean_B'], ddof=1)
        cov_AB = np.cov(question_means['mean_A'], question_means['mean_B'], ddof=1)[0, 1]
        omega_squared = var_A + var_B - 2 * cov_AB
        method = "simple"
    
    # Calculate sigma_squared terms if multiple draws available
    if has_multiple_draws:
        sigma_A_squared, sigma_B_squared = calculate_conditional_variances(
            df, question_id_col, question_draw_col, question_type_col, 
            score1_col, score2_col, is_clustered
        )
    else:
        sigma_A_squared = 0.0
        sigma_B_squared = 0.0
        print("Note: Only single draw per question, so σ²_A = σ²_B = 0")
    
    # Compile results
    results = {
        'omega_squared': omega_squared,
        'sigma_A_squared': sigma_A_squared,
        'sigma_B_squared': sigma_B_squared,
        'method': method,
        'is_clustered': is_clustered,
        'has_multiple_draws': has_multiple_draws,
        'n_questions': n_total_questions,
        'n_question_types': n_question_types,
        'n_draws_per_question': n_draws_per_question,
        'var_A': np.var(question_means['mean_A'], ddof=1),
        'var_B': np.var(question_means['mean_B'], ddof=1),
        'cov_AB': np.cov(question_means['mean_A'], question_means['mean_B'], ddof=1)[0, 1]
    }
    
    print(f"Results using {method} method:")
    print(f"  ω² = {omega_squared:.6f}")
    print(f"  σ²_A = {sigma_A_squared:.6f}")
    print(f"  σ²_B = {sigma_B_squared:.6f}")
    
    return results

def calculate_clustered_omega_squared(question_means: pd.DataFrame, 
                                    question_type_col: str,
                                    overall_mean_A: float,
                                    overall_mean_B: float) -> float:
    """Calculate ω² using clustered variance/covariance formulas."""
    
    n = len(question_means)
    
    # Calculate clustered variances and covariance
    var_A_clustered = 0
    var_B_clustered = 0
    cov_AB_clustered = 0
    
    for question_type in question_means[question_type_col].unique():
        cluster_data = question_means[question_means[question_type_col] == question_type]
        
        for _, row_i in cluster_data.iterrows():
            for _, row_j in cluster_data.iterrows():
                # Variance A component
                var_A_clustered += (row_i['mean_A'] - overall_mean_A) * (row_j['mean_A'] - overall_mean_A)
                
                # Variance B component
                var_B_clustered += (row_i['mean_B'] - overall_mean_B) * (row_j['mean_B'] - overall_mean_B)
                
                # Covariance component
                cov_AB_clustered += (row_i['mean_A'] - overall_mean_A) * (row_j['mean_B'] - overall_mean_B)
    
    var_A_clustered /= n
    var_B_clustered /= n
    cov_AB_clustered /= n
    
    omega_squared = var_A_clustered + var_B_clustered - 2 * cov_AB_clustered
    
    return omega_squared

def calculate_conditional_variances(df: pd.DataFrame,
                                  question_id_col: str,
                                  question_draw_col: str, 
                                  question_type_col: str,
                                  score1_col: str,
                                  score2_col: str,
                                  is_clustered: bool) -> Tuple[float, float]:
    """Calculate σ²_A and σ²_B (conditional variances)."""
    
    n_total = df[question_id_col].nunique()
    
    if is_clustered:
        return calculate_clustered_conditional_variances(
            df, question_id_col, question_draw_col, question_type_col,
            score1_col, score2_col, n_total
        )
    else:
        return calculate_simple_conditional_variances(
            df, question_id_col, score1_col, score2_col
        )

def calculate_simple_conditional_variances(df: pd.DataFrame,
                                         question_id_col: str,
                                         score1_col: str,
                                         score2_col: str) -> Tuple[float, float]:
    """Calculate simple (non-clustered) conditional variances."""
    
    # Calculate conditional variance for each question
    question_variances = df.groupby(question_id_col).agg({
        score1_col: lambda x: np.var(x, ddof=1) if len(x) > 1 else 0,
        score2_col: lambda x: np.var(x, ddof=1) if len(x) > 1 else 0
    })
    
    # Expected conditional variances
    sigma_A_squared = question_variances[score1_col].mean()
    sigma_B_squared = question_variances[score2_col].mean()
    
    return sigma_A_squared, sigma_B_squared

def calculate_clustered_conditional_variances(df: pd.DataFrame,
                                            question_id_col: str,
                                            question_draw_col: str,
                                            question_type_col: str,
                                            score1_col: str,
                                            score2_col: str,
                                            n_total: int) -> Tuple[float, float]:
    """Calculate clustered conditional variances using the paper's formula."""
    
    # Calculate conditional means for each question
    question_means = df.groupby([question_id_col, question_type_col]).agg({
        score1_col: 'mean',
        score2_col: 'mean'
    }).reset_index()
    
    # Merge back to get conditional means for each observation
    df_with_means = df.merge(question_means, on=[question_id_col, question_type_col], 
                            suffixes=('', '_mean'))
    
    K = df.groupby(question_id_col)[question_draw_col].nunique().max()
    
    # Calculate clustered conditional variances
    sigma_A_clustered = 0
    sigma_B_clustered = 0
    
    for question_type in df_with_means[question_type_col].unique():
        cluster_data = df_with_means[df_with_means[question_type_col] == question_type]
        
        for _, row_i in cluster_data.iterrows():
            for _, row_j in cluster_data.iterrows():
                # Calculate epsilon values (residuals from conditional means)
                epsilon_A_i = row_i[score1_col] - row_i[f'{score1_col}_mean']
                epsilon_A_j = row_j[score1_col] - row_j[f'{score1_col}_mean']
                epsilon_B_i = row_i[score2_col] - row_i[f'{score2_col}_mean']
                epsilon_B_j = row_j[score2_col] - row_j[f'{score2_col}_mean']
                
                sigma_A_clustered += epsilon_A_i * epsilon_A_j
                sigma_B_clustered += epsilon_B_i * epsilon_B_j
    
    sigma_A_clustered /= (n_total * (K - 1))
    sigma_B_clustered /= (n_total * (K - 1))
    
    return sigma_A_clustered, sigma_B_clustered

def create_sample_data() -> pd.DataFrame:
    """Create sample data to demonstrate the function."""
    np.random.seed(42)
    
    data = []
    question_id = 0
    
    # Create data with 2 question types, 10 questions each, 3 draws per question
    for question_type in ['reading_comp', 'math']:
        for q in range(10):
            question_id += 1
            base_difficulty_A = np.random.uniform(0.4, 0.8)
            base_difficulty_B = base_difficulty_A + np.random.normal(0, 0.1)
            
            for draw in range(1, 4):  # 3 draws per question
                score_A = np.random.binomial(1, base_difficulty_A)
                score_B = np.random.binomial(1, np.clip(base_difficulty_B, 0, 1))
                
                data.append({
                    'question_id': f'q_{question_id}',
                    'question_draw': draw,
                    'question_type': question_type,
                    'scorer1_id': 'model_A',
                    'scorer2_id': 'model_B', 
                    'score1': score_A,
                    'score2': score_B
                })
    
    return pd.DataFrame(data)

<div align="center">

## 2. Define functions for power calcs with grid search
----

</div>


In [ ]:
def calculate_sample_size(omega_squared: float, 
                        sigma_A_squared: float, 
                          sigma_B_squared: float, 
                          z_alpha_2: float, 
                          z_beta: float, 
                          K_A: int, 
                          K_B: int, 
                          delta: float) -> float:
    """
    Calculate required sample size

    Parameters:
    -----------
    omega_squared : float
        Variance of score differences (ω²)
    sigma_A_squared : float  
        Expected conditional variance for model A (σ²_A)
    sigma_B_squared : float
        Expected conditional variance for model B (σ²_B)
    z_alpha_2 : float
        Critical value for α/2 (two-tailed test)
    z_beta : float
        Critical value for β (Type II error)
    K_A : int
        Number of samples per question for model A
    K_B : int
        Number of samples per question for model B
    delta : float
        Minimum detectable effect size
        
    Returns:
    --------
    n : float
        Required number of questions
    """
    numerator = (z_alpha_2 + z_beta)**2 * (omega_squared + sigma_A_squared/K_A + sigma_B_squared/K_B)
    denominator = delta**2
    
    return numerator / denominator

def calculate_minimum_detectable_effect(omega_squared, sigma_A_squared, sigma_B_squared,
                                      z_alpha_2, z_beta, K_A, K_B, n):
    """
    Calculate minimum detectable effect for given sample size.
    Inverted version of the sample size formula.
    
    Returns:
    --------
    delta : float
        Minimum detectable effect size
    """
    variance_term = omega_squared + sigma_A_squared/K_A + sigma_B_squared/K_B
    delta = (z_alpha_2 + z_beta) * np.sqrt(variance_term / n)
    
    return delta

def create_power_analysis_grid():
    """
    Create a comprehensive grid of parameters for power analysis.
    
    Returns:
    --------
    dict : Parameter grid with all combinations
    """
    
    # Standard significance levels and their z-values (two sided)
    significance_levels = {
        'alpha_0.01': {'alpha': 0.01, 'z_alpha_2': stats.norm.ppf(1 - 0.01/2)},  # z ≈ 2.576
        'alpha_0.05': {'alpha': 0.05, 'z_alpha_2': stats.norm.ppf(1 - 0.05/2)},  # z ≈ 1.960  
        'alpha_0.10': {'alpha': 0.10, 'z_alpha_2': stats.norm.ppf(1 - 0.10/2)},  # z ≈ 1.645
    }
    
    # Standard power levels and their z-values
    power_levels = {
        'power_0.80': {'beta': 0.20, 'power': 0.80, 'z_beta': stats.norm.ppf(1 - 0.20)},  # z ≈ 0.842
        'power_0.85': {'beta': 0.15, 'power': 0.85, 'z_beta': stats.norm.ppf(1 - 0.15)},  # z ≈ 1.036
        'power_0.90': {'beta': 0.10, 'power': 0.90, 'z_beta': stats.norm.ppf(1 - 0.10)},  # z ≈ 1.282
        'power_0.95': {'beta': 0.05, 'power': 0.95, 'z_beta': stats.norm.ppf(1 - 0.05)},  # z ≈ 1.645
    }
    
    # Sampling strategies (K values)
    K_values = [1, 2, 3, 5, 10]  # Number of samples per question
    
    # Create the parameter grid
    grid = {
        'significance_levels': significance_levels,
        'power_levels': power_levels, 
        'K_values': K_values
    }
    
    return grid

def create_delta_grid(var_x_A, var_x_B, n_points=10):
    """
    Create a grid of delta values based on the standard deviations of x_A and x_B.
    
    Parameters:
    -----------
    var_x_A : float
        Variance of conditional means for model A
    var_x_B : float  
        Variance of conditional means for model B
    n_points : int
        Number of delta values to generate
        
    Returns:
    --------
    list : Delta values ranging from small to large effects
    """
    # Calculate standard deviations
    sd_x_A = np.sqrt(var_x_A)
    sd_x_B = np.sqrt(var_x_B)
    avg_sd = (sd_x_A + sd_x_B) / 2
    
    # Create delta values as fractions of the average standard deviation
    # Small effect: 0.1 * SD, Medium: 0.5 * SD, Large: 1.0 * SD
    effect_sizes = np.linspace(0.05, 1.5, n_points)  # From 5% to 150% of SD
    delta_values = effect_sizes * avg_sd
    
    return delta_values.tolist()

def run_power_analysis_grid(omega_squared, sigma_A_squared, sigma_B_squared,
                           var_x_A, var_x_B, max_n=10000):
    """
    Run comprehensive power analysis across parameter grid.
    
    Parameters:
    -----------
    omega_squared : float
        Estimated ω² from your data
    sigma_A_squared : float
        Estimated σ²_A from your data  
    sigma_B_squared : float
        Estimated σ²_B from your data
    var_x_A : float
        Variance of conditional means for model A
    var_x_B : float
        Variance of conditional means for model B
    max_n : int
        Maximum sample size to consider practical
        
    Returns:
    --------
    pd.DataFrame : Results of grid search
    """
    
    # Create parameter grids
    param_grid = create_power_analysis_grid()
    delta_values = create_delta_grid(var_x_A, var_x_B)
    
    results = []
    
    # Grid search over all combinations
    for sig_name, sig_params in param_grid['significance_levels'].items():
        for pow_name, pow_params in param_grid['power_levels'].items():
            for K in param_grid['K_values']:
                for delta in delta_values:
                    
                    # Calculate required sample size
                    n_required = calculate_sample_size(
                        omega_squared, sigma_A_squared, sigma_B_squared,
                        sig_params['z_alpha_2'], pow_params['z_beta'],
                        K, K, delta  # Assuming K_A = K_B = K
                    )
                    
                    # Calculate effect size relative to average SD
                    avg_sd = (np.sqrt(var_x_A) + np.sqrt(var_x_B)) / 2
                    relative_effect = delta / avg_sd if avg_sd > 0 else np.nan
                    
                    # Store results
                    results.append({
                        'alpha': sig_params['alpha'],
                        'power': pow_params['power'],
                        'beta': pow_params['beta'],
                        'K': K,
                        'delta': delta,
                        'delta_relative_sd': relative_effect,
                        'n_required': n_required,
                        'feasible': n_required <= max_n,
                        'z_alpha_2': sig_params['z_alpha_2'],
                        'z_beta': pow_params['z_beta']
                    })
    
    return pd.DataFrame(results)

def analyze_power_results(results_df, top_n=10):
    """
    Analyze and summarize power analysis results.
    
    Parameters:
    -----------
    results_df : pd.DataFrame
        Results from run_power_analysis_grid()
    top_n : int
        Number of top recommendations to show
        
    Returns:
    --------
    dict : Analysis summary
    """
    
    # Filter to feasible options
    feasible = results_df[results_df['feasible']].copy()
    
    if len(feasible) == 0:
        print("⚠️  No feasible options found! Consider:")
        print("   - Increasing max_n")
        print("   - Accepting larger effect sizes")
        print("   - Reducing power requirements")
        return None
    
    # Find optimal configurations
    print("🎯 POWER ANALYSIS SUMMARY")
    print("=" * 50)
    
    # Most efficient (smallest n) for different effect sizes
    print("\n📊 Most Efficient Configurations (Smallest Sample Size):")
    efficient = feasible.nsmallest(top_n, 'n_required')[
        ['alpha', 'power', 'K', 'delta_relative_sd', 'n_required']
    ].round(3)
    print(efficient.to_string(index=False))
    
    # Different effect size categories
    small_effect = feasible[feasible['delta_relative_sd'] <= 0.3]
    medium_effect = feasible[(feasible['delta_relative_sd'] > 0.3) & (feasible['delta_relative_sd'] <= 0.7)]
    large_effect = feasible[feasible['delta_relative_sd'] > 0.7]
    
    print(f"\n📈 Sample Size Ranges:")
    print(f"Small effects (≤0.3 SD): {small_effect['n_required'].min():.0f} - {small_effect['n_required'].max():.0f} questions")
    print(f"Medium effects (0.3-0.7 SD): {medium_effect['n_required'].min():.0f} - {medium_effect['n_required'].max():.0f} questions")  
    print(f"Large effects (>0.7 SD): {large_effect['n_required'].min():.0f} - {large_effect['n_required'].max():.0f} questions")
    
    # Impact of K (sampling strategy)
    print(f"\n🔄 Impact of Multiple Sampling (K):")
    k_impact = feasible.groupby('K')['n_required'].agg(['mean', 'min']).round(0)
    print(k_impact)
    
    return {
        'feasible_options': len(feasible),
        'min_n': feasible['n_required'].min(),
        'max_n': feasible['n_required'].max(),
        'recommended': efficient.iloc[0].to_dict() if len(efficient) > 0 else None
    }

def plot_power_analysis(results_df, save_path=None):
    """
    Create visualizations of power analysis results.
    """
    feasible = results_df[results_df['feasible']].copy()
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Power Analysis Results', fontsize=16, fontweight='bold')
    
    # 1. Sample size vs effect size
    ax1 = axes[0, 0]
    for alpha in feasible['alpha'].unique():
        for power in feasible['power'].unique():
            subset = feasible[(feasible['alpha'] == alpha) & 
                            (feasible['power'] == power) & 
                            (feasible['K'] == 1)]
            if len(subset) > 0:
                ax1.plot(subset['delta_relative_sd'], subset['n_required'], 
                        'o-', label=f'α={alpha}, power={power}', alpha=0.7)
    
    ax1.set_xlabel('Effect Size (relative to SD)')
    ax1.set_ylabel('Required Sample Size')
    ax1.set_title('Sample Size vs Effect Size (K=1)')
    ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax1.grid(True, alpha=0.3)
    
    # 2. Impact of K
    ax2 = axes[0, 1]
    for K in sorted(feasible['K'].unique()):
        subset = feasible[(feasible['K'] == K) & 
                         (feasible['alpha'] == 0.05) & 
                         (feasible['power'] == 0.8)]
        if len(subset) > 0:
            ax2.plot(subset['delta_relative_sd'], subset['n_required'], 
                    'o-', label=f'K={K}', alpha=0.7)
    
    ax2.set_xlabel('Effect Size (relative to SD)')
    ax2.set_ylabel('Required Sample Size')
    ax2.set_title('Impact of Multiple Sampling (α=0.05, power=0.8)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Heatmap of feasible configurations
    ax3 = axes[1, 0]
    pivot = feasible.pivot_table(values='n_required', 
                               index=['alpha', 'power'], 
                               columns='K', 
                               aggfunc='mean')
    sns.heatmap(pivot, annot=True, fmt='.0f', cmap='viridis_r', ax=ax3)
    ax3.set_title('Average Sample Size by Configuration')
    
    # 4. Effect size recommendations
    ax4 = axes[1, 1]
    bins = [0, 0.3, 0.7, 1.0, 2.0]
    labels = ['Small\n(≤0.3 SD)', 'Medium\n(0.3-0.7 SD)', 'Large\n(0.7-1.0 SD)', 'Very Large\n(>1.0 SD)']
    feasible['effect_category'] = pd.cut(feasible['delta_relative_sd'], bins=bins, labels=labels, include_lowest=True)
    
    effect_summary = feasible.groupby('effect_category')['n_required'].agg(['mean', 'min', 'max'])
    effect_summary.plot(kind='bar', ax=ax4)
    ax4.set_title('Sample Size by Effect Category')
    ax4.set_ylabel('Required Sample Size')
    ax4.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    
    plt.show()

<div align="center">

## 3. Generate sample data
----

</div>

In [ ]:
def create_sample_data(n_question_types=2, n_questions_per_type=10, n_draws=3, 
                      add_correlation=True, seed=42):
    """
    Create sample data in long format for LLM evaluation analysis.
    
    Parameters:
    -----------
    n_question_types : int
        Number of question types/clusters (default: 2)
    n_questions_per_type : int
        Number of questions per type (default: 10)
    n_draws : int
        Number of draws/samples per question (default: 3)
    add_correlation : bool
        Whether to add correlation between scorers (default: True)
    seed : int
        Random seed for reproducibility (default: 42)
        
    Returns:
    --------
    pd.DataFrame : Long format data with columns:
        - question_id: Unique identifier for each question
        - question_draw: Draw number (1, 2, 3, ...)
        - question_type: Type/cluster of question
        - scorer1_id: Always 'model_A'
        - scorer2_id: Always 'model_B'
        - score1: Score from model A
        - score2: Score from model B
    """
    np.random.seed(seed)
    
    data = []
    question_id = 0
    
    # Question types (clusters)
    question_types = [f'type_{i}' for i in range(n_question_types)]
    
    for question_type in question_types:
        # Each question type has different base difficulty
        type_difficulty_A = np.random.uniform(0.3, 0.8)
        type_difficulty_B = np.random.uniform(0.3, 0.8)
        
        for q in range(n_questions_per_type):
            question_id += 1
            
            # Individual question difficulty (varies around type difficulty)
            base_difficulty_A = type_difficulty_A + np.random.normal(0, 0.15)
            base_difficulty_A = np.clip(base_difficulty_A, 0.05, 0.95)
            
            if add_correlation:
                # Model B performance correlates with Model A (similar questions are hard/easy for both)
                correlation_strength = 0.6
                base_difficulty_B = (correlation_strength * base_difficulty_A + 
                                   (1 - correlation_strength) * type_difficulty_B + 
                                   np.random.normal(0, 0.1))
            else:
                base_difficulty_B = type_difficulty_B + np.random.normal(0, 0.15)
            
            base_difficulty_B = np.clip(base_difficulty_B, 0.05, 0.95)
            
            # Generate multiple draws for this question
            for draw in range(1, n_draws + 1):
                # Add some randomness to each draw (conditional variance)
                prob_A = np.clip(base_difficulty_A + np.random.normal(0, 0.05), 0, 1)
                prob_B = np.clip(base_difficulty_B + np.random.normal(0, 0.05), 0, 1)
                
                score_A = np.random.binomial(1, prob_A)
                score_B = np.random.binomial(1, prob_B)
                
                data.append({
                    'question_id': f'q_{question_id:03d}',
                    'question_draw': draw,
                    'question_type': question_type,
                    'scorer1_id': 'model_A',
                    'scorer2_id': 'model_B',
                    'score1': score_A,
                    'score2': score_B
                })
    
    df = pd.DataFrame(data)

<div align="center">

## 4. Run
----

</div>

In [ ]:
sample_df = create_sample_data()
print("Sample data shape:", sample_df.shape)
print("\nFirst few rows:")
print(sample_df.head())
print("\n" + "="*50)

# Calculate state variables squared
results = calculate_omega_squared(sample_df)
print(f"Use ω² = {results['omega_squared']:.6f}")
print(f"Use σ²_A = {results['sigma_A_squared']:.6f}")  
print(f"Use σ²_B = {results['sigma_B_squared']:.6f}")

# Run power analysis grid search
results = run_power_analysis_grid(
    omega_squared=results['omega_squared'], 
    sigma_A_squared=results['sigma_A_squared'], 
    sigma_B_squared=results['sigma_B_squared'],
    var_x_A=results['var_x_A'], 
    var_x_B=results['var_x_B'],
    max_n=5000
    )

# Analyze results
summary = analyze_power_results(results)
plot_power_analysis(results)